In [72]:
import numpy as np
from numpy import mean,std
from scipy.stats import randint
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report,mean_squared_error
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold,train_test_split,cross_val_score,cross_validate

In [73]:
X= pd.read_csv('../data/processed/X_train', header=0, index_col=0)
X_final_test = pd.read_csv('../data/processed/X_final_test', header=0, index_col=0)
y = pd.read_csv('../data/processed/y_train', header=0, index_col=0)
y_final_test = pd.read_csv('../data/processed/y_final_test', header=0, index_col=0)

In [74]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

# Feature Scaling

In [75]:
# from sklearn.preprocessing import StandardScaler
#
# sc = StandardScaler()
# X_train = sc.fit_transform(X_train)
# X_test = sc.transform(X_test)

# Decision Tree Model Building

In [76]:
#without hyperparameter tuning
# Create Decision Tree classifer object
clf = DecisionTreeClassifier()

# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

# Making the Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)
print('Accuracy:', accuracy_score(y_test, y_pred)*100,'%')
print(classification_report(y_test,y_pred))

[[23  5]
 [ 7 13]]
Accuracy: 75.0 %
              precision    recall  f1-score   support

           0       0.77      0.82      0.79        28
           1       0.72      0.65      0.68        20

    accuracy                           0.75        48
   macro avg       0.74      0.74      0.74        48
weighted avg       0.75      0.75      0.75        48



In [77]:
#with hyperparameter tuning
# Create Decision Tree classifer object
clf = DecisionTreeClassifier(criterion="entropy", max_depth=3)

# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

# Model Accuracy, how often is the classifier correct?
print("Accuracy:",accuracy_score(y_test, y_pred))

Accuracy: 0.7708333333333334


# Cross Fold Validation

In [78]:
#cross fold validation without hyper tuning
cv = KFold(n_splits=10, random_state=42, shuffle=True)
clf = DecisionTreeClassifier()
print(cross_val_score(clf, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1).mean())

0.7304093567251463


In [93]:
#cross fold validation with hyper tuning
results_df = pd.DataFrame(index=['Decision Tree'],
                          columns=['F-score (cv)', 'Precision (cv)','Recall (cv)', 'Accuracy (cv)'])
cv = KFold(n_splits=10, random_state=42, shuffle=True)
clf = DecisionTreeClassifier(criterion="entropy", max_depth=3)
results=pd.DataFrame(cross_validate(clf, X_train, y_train, scoring = [ 'f1_macro', 'precision_macro', 'recall_macro', 'accuracy'], cv=cv, n_jobs=-1))
results_df.loc['Decision Tree',:] = results[['test_f1_macro',
       'test_precision_macro', 'test_recall_macro','test_accuracy']].mean().values

results_df

,F-score (cv),Precision (cv),Recall (cv),Accuracy (cv)
Decision Tree,0.733612,0.7539,0.749727,0.746491


# Hyperparameter tuning of DT using RandomizedSearchCV
In RandomizedSearchCV,not all hyperparameter values are used. Instead, a fixed number of hyperparameters are sampled from specified probability distributions.The following hyperparameter are chosen for tuning:

"max_depth":[3,None]
max_depth: The maximum depth of the tree.The higher value of maximum depth causes overfitting, and a lower value causes underfitting. So, we set 3 to none.
"min_samples_leaf":randint(1,20),
min_samples_leaf: The minimum number of samples required to be at a leaf node. We set random value to be chosen from 1 to 20.
"criterion":["gini","entropy"]
criterion: The function to measure the quality of a split. We set both gini and entropy to choose the best one for this decision tree model.

In [92]:
from sklearn.model_selection import RandomizedSearchCV

#paramters setup for tuning
param={"max_depth":[3,None],"min_samples_leaf":randint(1,20),"criterion":["gini","entropy"]}
#"min_samples_split":randint(1,40)
#"min_samples_leaf":randint(1,20)

#instatiate decision tree classifier
clf = DecisionTreeClassifier()

#instatiate RandomizedSearchCV
clf_cv=RandomizedSearchCV(clf,param,cv=5)

#fitting the model
clf_cv.fit(X_train,y_train)

#print the tuned parameters and score
print("Best: %f using %s" % (clf_cv.best_score_.mean(), clf_cv.best_params_))
means = clf_cv.cv_results_['mean_test_score']
stds = clf_cv.cv_results_['std_test_score']
params = clf_cv.cv_results_['params']
# for mean, stdev, param in zip(means, stds, params):
#     print("%f (%f) with: %r" % (mean, stdev, param))
#printing the confusion metrix
y_pred=clf_cv.predict(X_test)
print(confusion_matrix(y_test,y_pred))


Best: 0.825605 using {'criterion': 'entropy', 'max_depth': None, 'min_samples_leaf': 9}
[[24  4]
 [ 6 14]]


# Building the model using Bagging Classifier

In [81]:
from sklearn.ensemble import BaggingClassifier

bagging_model = BaggingClassifier(
base_estimator=DecisionTreeClassifier(),
n_estimators=100,
max_samples=0.8,
bootstrap=True,
oob_score=True,
random_state=0
)
bagging_model.fit(X_train,y_train)
print(bagging_model.oob_score_)


0.8148148148148148


C:\Users\Asus\PycharmProjects\Heart-Disease-Machine-Learning-Exploration\venv\lib\site-packages\sklearn\ensemble\_bagging.py:719: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
